In [ ]:
ls

In [ ]:
pwd

In [ ]:
import pandas as pd

In [2]:
import findspark
findspark.init('/Users/awf/notebook/spark-3.3.1-bin-hadoop3')

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql import SparkSession

appName = "Spark - Setting Log Level"
master = "local"

# Create Spark session
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")

In [ ]:
"""
IllegalArgumentException: requirement failed: Supplied level newLevel did not match one of: 

ALL,DEBUG,ERROR,FATAL,TRACE,WARN,INFO,OFF

"""

In [11]:
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.appName('UDF').getOrCreate()

#spark.sparkContext.setLogLevel("OFF")

spark

In [12]:
#Nous créons également la variable sc qui est un parkContext issue de la variable spark*
sc = spark.sparkContext
sc.setLogLevel("OFF")

In [ ]:
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import udf

from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [22]:
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

@pandas_udf("col1 string, col2 long")
def func(s1: pd.Series, s2: pd.Series, s3: pd.DataFrame) -> pd.DataFrame:
    s3['col2'] = s1 + s2.str.len()
    return s3

In [23]:
# Create a Spark DataFrame that has three columns including a struct column.
df = spark.createDataFrame(
    [[1, "a string", ("a nested string",)]],
    "long_col long, string_col string, struct_col struct<col1:string>")
df.printSchema()

root
 |-- long_col: long (nullable = true)
 |-- string_col: string (nullable = true)
 |-- struct_col: struct (nullable = true)
 |    |-- col1: string (nullable = true)



In [24]:
df.show()

+--------+----------+-----------------+
|long_col|string_col|       struct_col|
+--------+----------+-----------------+
|       1|  a string|{a nested string}|
+--------+----------+-----------------+



In [25]:
df.select(func("long_col", "string_col", "struct_col")).printSchema()

root
 |-- func(long_col, string_col, struct_col): struct (nullable = true)
 |    |-- col1: string (nullable = true)
 |    |-- col2: long (nullable = true)



In [26]:
df.select(func("long_col", "string_col", "struct_col")).show()

+--------------------------------------+
|func(long_col, string_col, struct_col)|
+--------------------------------------+
|                  {a nested string, 9}|
+--------------------------------------+



In [19]:
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

@pandas_udf("string")
def to_upper(s: pd.Series) -> pd.Series:
    return s.str.upper()

df = spark.createDataFrame([("John Doe",)], ("name",))
df.show()

+--------+
|    name|
+--------+
|John Doe|
+--------+



In [20]:
df.select(to_upper("name")).show()

+--------------+
|to_upper(name)|
+--------------+
|      JOHN DOE|
+--------------+



In [28]:
from typing import Iterator

@pandas_udf("long")
def calculate(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    # Do some expensive initialization with a state
    state = very_expensive_initialization()
    for x in iterator:
        # Use that state for whole iterator.
        yield calculate_with_state(x, state)

In [30]:
from typing import Iterator
@pandas_udf("long")
def plus_one(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    for s in iterator:
        yield s + 1

df = spark.createDataFrame(pd.DataFrame([1, 2, 3], columns=["v"]))
df.select(plus_one(df.v)).show()

/Users/awf/notebook/spark-3.3.1-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/awf/notebook/spark-3.3.1-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-----------+
|plus_one(v)|
+-----------+
|          2|
|          3|
|          4|
+-----------+




[Stage 22:===========================================>              (3 + 1) / 4]



In [31]:
# Import
from pyspark.sql import SparkSession

# Create SparkSession
#spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

# Prepare Data
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

# Create DataFrame
df = spark.createDataFrame(data=data,schema=columns)
df.show()

# imports
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
import pandas as pd

# create pandas_udf
@pandas_udf(StringType())
def to_upper(s: pd.Series) -> pd.Series:
    return s.str.upper()

# Using UDF with select()
df.select("Seqno","Name",to_upper("Name")).show()

# Using UDF with withColumn()
df.withColumn("upper_col",to_upper("Name")).show()

+-----+------------+
|Seqno|        Name|
+-----+------------+
|    1|  john jones|
|    2|tracey smith|
|    3| amy sanders|
+-----+------------+



+-----+------------+--------------+
|Seqno|        Name|to_upper(Name)|
+-----+------------+--------------+
|    1|  john jones|    JOHN JONES|
|    2|tracey smith|  TRACEY SMITH|
|    3| amy sanders|   AMY SANDERS|
+-----+------------+--------------+

+-----+------------+------------+
|Seqno|        Name|   upper_col|
+-----+------------+------------+
|    1|  john jones|  JOHN JONES|
|    2|tracey smith|TRACEY SMITH|
|    3| amy sanders| AMY SANDERS|
+-----+------------+------------+



In [32]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [34]:
import numpy as np
import pandas as pd


# Generate a Pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))
pdf.head()

,0,1,2
0,0.818666,0.513061,0.735572
1,0.272248,0.189142,0.421213
2,0.559945,0.864359,0.949797
3,0.909914,0.992102,0.741554
4,0.679805,0.968403,0.202535


In [38]:
# Create a Spark DataFrame from a Pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

df.head(5)

[Row(0=0.8186662468647464, 1=0.5130609817604163, 2=0.7355715729334559),
 Row(0=0.2722479066492186, 1=0.1891415247185636, 2=0.4212134102765188),
 Row(0=0.5599445550106983, 1=0.8643593339869255, 2=0.9497969959680351),
 Row(0=0.9099139217992909, 1=0.9921020177135674, 2=0.7415543876581406),
 Row(0=0.6798045639833243, 1=0.9684033493858719, 2=0.20253525116862536)]

In [41]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()

In [42]:
result_pdf

,0,1,2
0,0.818666,0.513061,0.735572
1,0.272248,0.189142,0.421213
2,0.559945,0.864359,0.949797
3,0.909914,0.992102,0.741554
4,0.679805,0.968403,0.202535
...,...,...,...
95,0.357483,0.130592,0.058661
96,0.358264,0.932145,0.234140
97,0.678177,0.875813,0.281925
98,0.516185,0.369003,0.524335


In [44]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the function and create the UDF
def multiply_func(a, b):
    return a * b

In [48]:
multiply = pandas_udf(multiply_func, returnType=LongType())

In [49]:
# The function for a pandas_udf should be able to execute with local Pandas data
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))

0    1
1    4
2    9
dtype: int64


In [53]:
# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

/Users/awf/notebook/spark-3.3.1-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


In [52]:
# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()

+-------------------+
|multiply_func(x, x)|
+-------------------+
|                  1|
|                  4|
|                  9|
+-------------------+



In [59]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf, struct, PandasUDFType

pdf = pd.DataFrame([1, 2, 3], columns=["x"])
pdf

,x
0,1
1,2
2,3


In [83]:
df = spark.createDataFrame(pdf)
df.show()

+---+
|  x|
+---+
|  1|
|  2|
|  3|
+---+



In [93]:
df.head(3)

[Row(x=1), Row(x=2), Row(x=3)]

In [84]:
df.printSchema()

root
 |-- x: long (nullable = true)



In [89]:
from pyspark.sql.types import StringType, IntegerType, LongType

# When the UDF is called with a single column that is not StructType,
# the input to the underlying function is an iterator of pd.Series.
@pandas_udf("long", PandasUDFType.SCALAR_ITER)
#@pandas_udf(LongType, PandasUDFType.SCALAR_ITER)
#@pandas_udf("float", PandasUDFType.SCALAR_ITER)
#@pandas_udf(IntegerType(), PandasUDFType.SCALAR_ITER)
def plus_one(batch_iter):
    for x in batch_iter:
        yield x + 1

In [90]:
df.select(plus_one(col("x"))).show()

+-----------+
|plus_one(x)|
+-----------+
|          2|
|          3|
|          4|
+-----------+



In [91]:
# When the UDF is called with more than one columns,
# the input to the underlying function is an iterator of pd.Series tuple.
@pandas_udf("long", PandasUDFType.SCALAR_ITER)
def multiply_two_cols(batch_iter):
    for a, b in batch_iter:
        yield a * b

df.select(multiply_two_cols(col("x"), col("x"))).show()


+-----------------------+
|multiply_two_cols(x, x)|
+-----------------------+
|                      1|
|                      4|
|                      9|
+-----------------------+



In [95]:
# When the UDF is called with a single column that is StructType,
# the input to the underlying function is an iterator of pd.DataFrame.
@pandas_udf("long", PandasUDFType.SCALAR_ITER)
def multiply_two_nested_cols(pdf_iter):
    for pdf in pdf_iter:
        yield pdf["a"] * pdf["b"]

df.select(
    multiply_two_nested_cols(
        struct(col("x").alias("a"), col("x").alias("b"))
    ).alias("y")
).show()

+---+
|  y|
+---+
|  1|
|  4|
|  9|
+---+



In [107]:
large_broadcast = sc.broadcast(range(10))

In [108]:
large_broadcast.value

range(0, 10)

In [105]:
#large_broadcast.value.collect()

In [109]:
sc.parallelize([0, 0]).flatMap(lambda x: large_broadcast.value).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [111]:
large_broadcast.unpersist()

In [112]:
large_broadcast.value

range(0, 10)

In [113]:
broadcastVar = sc.broadcast(Array(0, 1, 2, 3))
broadcastVar.value

NameError: name 'Array' is not defined

In [116]:

states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]



data

[('James', 'Smith', 'USA', 'CA'),
 ('Michael', 'Rose', 'USA', 'NY'),
 ('Robert', 'Williams', 'USA', 'CA'),
 ('Maria', 'Jones', 'USA', 'FL')]

In [122]:
rdd = spark.sparkContext.parallelize(data)
rdd

ParallelCollectionRDD[250] at readRDDFromFile at PythonRDD.scala:274

In [118]:
def state_convert(code):
    return broadcastStates.value[code]

result = rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).collect()
print(result)

[('James', 'Smith', 'USA', 'California'), ('Michael', 'Rose', 'USA', 'New York'), ('Robert', 'Williams', 'USA', 'California'), ('Maria', 'Jones', 'USA', 'Florida')]
